In [2]:
import os
import shutil
import random
import pathlib
import pandas as pd

In [17]:
data = pd.read_csv(r"C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\DL\RedesConvolucionales\Mauna_Kea\MultiClassifier\TrainingSet_20aimVO.csv")

In [4]:
data.head()

,image_filename,class_number
0,im_4_0.png,0
1,im_21_0.png,0
2,im_9_0.png,0
3,im_8_0.png,0
4,im_15_0.png,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9446 entries, 0 to 9445
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   image_filename  9446 non-null   object
 1   class_number    9446 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 147.7+ KB


In [6]:
data['class_number'].value_counts()

3    3594
1    3177
0    1469
2    1206
Name: class_number, dtype: int64

In [7]:
#Spliting and Copying into Train-Test Folders
target_dir = r'C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\DL\RedesConvolucionales\Mauna_Kea\MultiClassifier\ClassifiedImages\\'
image_path = r'C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\DL\RedesConvolucionales\Mauna_Kea\MultiClassifier\TrainingSetImagesDir\\'

count = 0
for image_split in [0, 1, 2, 3]:
    labels = list(data[(data['class_number']==image_split)]['image_filename'])
    for label in labels:
        to_dir = pathlib.Path(str(target_dir) + str(image_split) + '/' + str(label))
        if not to_dir.is_dir():
            to_dir.parent.mkdir(parents=True, exist_ok=True)
        from_dir = pathlib.Path(image_path + label)
        shutil.copy2(from_dir, to_dir)
        count += 1

print(f'{count:.0f} images copied ')

9446 images copied 


In [18]:
# Setup data paths
data_path = r'C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\DL\RedesConvolucionales\Mauna_Kea\MultiClassifier\ClassifiedImages\\'

# Create function to split randomly in Train and Test
def train_val_split(image_path=data_path,
               data_splits=['Train', 'Val'], 
               target_classes = [0, 1, 2, 3],
               split=0.05,
               seed=42):
    random.seed(42)
    label_splits = {}
    
    # Get labels
    for data_split in data_splits:
        print(f"[INFO] Creating image split for: {data_split}...")
        image_paths = []
        for target in target_classes:
            labels = list(data[(data['class_number']==target)]['image_filename'])
            sample = round((1-split)*len(labels))
            print(f"[INFO] Getting random set of {sample} images for {data_split}-{target} ...")
            if data_split == 'Train':
                sampled_images = random.sample(labels, k=sample)
            elif data_split == 'Val':
                sampled_images = labels
            image_paths.append([pathlib.Path(str(image_path + str(target) + '/' + sample_image)) for sample_image in sampled_images])
            data.drop(data[data['image_filename'].isin(sampled_images)].index, inplace=True)
            print(f'Left images: {len(data)}')
        image_path_flat = [item for sublist in image_paths for item in sublist]
        label_splits[data_split] = image_path_flat
    
    print('\n','Finish splitting!')

    return label_splits

In [19]:
label_splits = train_val_split(split=0.10)

[INFO] Creating image split for: Train...
[INFO] Getting random set of 1322 images for Train-0 ...
Left images: 8124
[INFO] Getting random set of 2859 images for Train-1 ...
Left images: 5265
[INFO] Getting random set of 1085 images for Train-2 ...
Left images: 4180
[INFO] Getting random set of 3235 images for Train-3 ...
Left images: 945
[INFO] Creating image split for: Val...
[INFO] Getting random set of 132 images for Val-0 ...
Left images: 798
[INFO] Getting random set of 286 images for Val-1 ...
Left images: 480
[INFO] Getting random set of 109 images for Val-2 ...
Left images: 359
[INFO] Getting random set of 323 images for Val-3 ...
Left images: 0

 Finish splitting!


In [20]:
label_splits['Train'][0:10]

[WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/DL/RedesConvolucionales/Mauna_Kea/MultiClassifier/ClassifiedImages/0/im_277_31.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/DL/RedesConvolucionales/Mauna_Kea/MultiClassifier/ClassifiedImages/0/im_98_4.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/DL/RedesConvolucionales/Mauna_Kea/MultiClassifier/ClassifiedImages/0/im_46_1.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/DL/RedesConvolucionales/Mauna_Kea/MultiClassifier/ClassifiedImages/0/im_173_6.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/DL/RedesConvolucionales/Mauna_Kea/MultiClassifier/ClassifiedImages/0/im_137_6.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/DL/RedesConvolucionales/Mauna_Kea/MultiClassifier/ClassifiedImages/0/im_56_6.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/DL/RedesConvolucionales/Mauna_

In [21]:
from collections import Counter
counter1 = Counter(label_splits['Train'])
counter2 = Counter(label_splits['Val'])
common_elements = counter1 & counter2
len(common_elements)

0

In [22]:
# Create target directory path
target_dir_name = r'C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\DL\RedesConvolucionales\Mauna_Kea\MultiClassifier\ClassifiedImages'
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories & Make the directories
target_dir = pathlib.Path(target_dir_name)
target_dir.mkdir(parents=True, exist_ok=True)


import shutil

count=0
for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parts[-2] / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(image_path, dest_dir)
        count += 1
        

print(f'{count:.0f} images moved')

Creating directory: 'C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\DL\RedesConvolucionales\Mauna_Kea\MultiClassifier\ClassifiedImages'
9446 images moved


In [23]:
# for image_split in label_splits.keys():
#     delete_dir = target_dir / image_split
#     files_in_dir = os.listdir(delete_dir)     # get list of files in the directory
#     for file in files_in_dir:                  # loop to delete each file in folder
#         os.remove(f'{delete_dir}/{file}')     # delete file
#     os.rmdir(image_path.parent)